# Assignment 2


#### 1. 复习上课内容以及复现课程代码

在本部分，你需要在复习上课内容和课程代码后，复现课程代码。 

#### 2. 回答以下理论题目

###       < 评阅点 >： 答案是否正确完整

###### 2.1 What conditions are required to make the BFS return the optimal solution ?

cost非负；搜索历史可以排序

##### 2.2 Is there a way to make DFS find the optimal solution ? (You may need to read some material about iterative DFS)

遍历所有路径

##### 2.3 In what conditions BFS is a better choice than DFS and vice versa ?

如果目标离树的根部不远，BFS会更好。
如果树很深并且目标少，BFS会更快。
如果树很宽，目标多，但位于树的深处，DFS会更好。
如果树深，则无论如何（例如迭代加深），都需要限制DFS的search深度。
DS通常用于游戏的模拟。

##### 2.4 When can we use machine learning ?

数据很多，呈现一定规律。

##### 2.5 What is the gradient of a function ?

导数的函数

##### 2.6 How can we find the maximum value of a function using the information of gradient ?

梯度函数接近0.

#### 3. 实践部分  寻找地铁路线

### < 评阅点 >  1: 爬虫爬取数据是否完整;  2:搜索算法是否正确

In this part, although we recommend you to use Beijing subway, you still can use the subway map of any cities that you are interested in. 

![title](img/ditu.png)

Please using the search policy to implement an agent. This agent receives two input, one is @param start station and the other is @param destination. Your agent should give the optimal route based on Beijing Subway system.

Deadline: 2019-May

#### Procedures

#### 1. Get data from web.  

Some tips: 

a. You might need this package: requests[https://2.python-requests.org/en/master/] to get webpages

In [6]:
import requests
r = requests.get('https://dt.8684.cn/nj', auth=('user', 'pass'))
r.status_code

200

In [7]:
r.headers['content-type']

'text/html'

In [8]:
r.encoding = 'utf-8'

In [9]:
html_doc = r.text

b.You might need to use Regular Expression and Beautiful Soap package to parse the webpages

In [10]:
import re
from bs4 import BeautifulSoup

In [235]:
soup = BeautifulSoup(html_doc, 'html.parser')
#soup.prettify()

##### 2. Preprocessing data

Some tips:

a. Find a suitable way to save the data you get from the web. (Note: The way you use to save the data should be able to be used to create the graph that your agent is going to explore)

In [33]:
starline_number = html_doc.find('南京地铁线路列表')
endline_number = html_doc.find('南京地铁5号线')
html_doc = html_doc[starline_number: endline_number] 

In [236]:
pattern = re.compile("南京地铁[\w][\w]?号线")
route1 = pattern.findall(html_doc)
pattern = re.compile("S?[\d][\d]?")
route2 = pattern.findall(''.join(route1))
route = ['line' + i for i in route2]
#route

In [155]:
station = re.findall('[\u4e00-\u9fa5]+', html_doc)

In [156]:
station_arrange = [[] for i in range(len(route))]

In [172]:
i = -1
for b in station[1:]:
    station_arrange[i].append(b)
    if b == '南京地铁':
        station_arrange[i].pop()
    if b == '号线':
        station_arrange[i].pop()
        i += 1

In [179]:
subway=[[] for i in range(len(route))]
for i in range(10):
    subway[i]= (str(route[i])) + '=' + '\'' + ' '.join(station_arrange[i]) + '\''+','
a = ''.join(subway)

In [225]:
def build_subway(**lines):
    """
    Input is build_subway(linename='station1 station2...'...)
    Ouput is a dictionary like {station:{neighbor1:line number,neighbor2:line number,...},station2:{...},...}
    """
    for key in lines.keys():
        value = lines[key]
        lines[key] = value.split()
    stations = set()
    for key in lines.keys():
        stations.update(set(lines[key]))
    graph = {}
    for station in stations:
        next_station = {}
        for key in lines:
            if station in lines[key]:
                value = lines[key]
                idx = value.index(station)
                if idx == 0:
                    next_station[value[1]] = key
                elif idx == len(value)-1:
                    next_station[value[idx-1]]=key
                else:
                    next_station[value[idx-1]] = key
                    next_station[value[idx+1]] = key
        graph[station] = next_station
    return graph

In [237]:
line_subway = build_subway(line1='迈皋桥 红山动物园 南京站 新模范马路 玄武门 鼓楼 珠江路 新街口 张府园 三山街 中华门 安德门 天隆寺 软件大道 花神庙 南京南站 双龙大道 河定桥 胜太路 百家湖 小龙湾 竹山路 天印大道 龙眠大道 江苏经贸学院 南京交院 中国药科大学',line2='油坊桥 雨润大街 元通 奥体东 兴隆大街 集庆门大街 云锦路 莫愁湖 汉中门 上海路 新街口 大行宫 西安门 明故宫 苜蓿园 下马坊 孝陵卫 钟灵街 马群 金马路 仙鹤门 学则路 仙林中心 羊山公园 南大仙林校区 经天路',line3='林场 星火路 东大成贤学院 泰冯路 天润城 柳洲东路 上元门 五塘广场 小市 南京站 南京林业大学 新庄 鸡鸣寺 浮桥 大行宫 常府街 夫子庙 武定门 雨花门 卡子门 大明路 明发广场 南京南站 宏运大道 胜太西路 天元西路 九龙湖 诚信大道 东大九龙湖校区 秣周东路',line4='仙林湖 西岗桦墅 孟北 东流 灵山 汇通路 金马路 苏宁总部 徐庄 聚宝山 王家湾 蒋王庙 岗子村 九华山 鸡鸣寺 鼓楼 云南路 南艺 二师 草场门 龙江',line10='安德门 小行 中胜 元通 奥体中心 梦都大街 绿博园 江心洲 临江 青奥体育公园 浦口万汇城 南京工业大学 龙华路 文德路 雨山路', lineS1='机场线 南京南站 翠屏山 河海大学佛城西路 吉印大道 正方中路 翔宇路北 翔宇路南 禄口机场 空港新城江宁',lineS3='宁和城际 高家冲 林山 桥林新城 石碛河 双垅 兰花塘 马骡圩 刘村 天保 高庙路 吴侯街 平良大街 永初路 油坊桥 贾西 春江路 铁心桥 景明佳园 南京南站',lineS7='宁溧城际 空港新城江宁 柘塘 空港新城溧水 群力 卧龙湖 溧水 中山湖 幸庄 无想山',lineS8='宁天城际 泰山新村 泰冯路 高新开发区 信息工程大学 卸甲甸 大厂 葛塘 长芦 化工园 六合开发区 龙池 雄州 凤凰山公园 方州广场 沈桥 八百桥 金牛湖',lineS9='宁高城际 翔宇路南 铜山 石湫 明觉 团结圩 高淳')
#line_subway

#### 3. Build the search agent

Build the search agent based ont he graph you built.

In [227]:
def search_1(graph,start,destination):
    pathes = [[start]]  # list 用来存储待搜索路径
    visited = set() # set用来存储已搜索的节点
    
    while pathes:
        path = pathes.pop(0)  #提取第一条路径
        froniter = path[-1]   #提取即将要探索的节点
        
        if froniter in visited: continue  #检查如果该点已经探索过 则不用再探索
            
        successsors = graph[froniter]
        
        for city in successsors:      #遍历子节点
            if city in path: continue  # check loop #检查会不会形成环
            
            new_path = path+[city]
            
            pathes.append(new_path)  #bfs     #将新路径加到list里面
            #pathes = [new_path] + pathes #dfs
            
            if city == destination:  #检查目的地是不是已经搜索到了
                return new_path
        visited.add(froniter)

In [230]:
def shorter_path(start, goal):
    """
    without consideration of the change times fina shortest path
    """
    if start == goal:
        return [start]
    explored = set() 
    queue = [ [start] ] 
    while queue:
        path = queue.pop(0)
        s = path[-1]
        for state, action in line_subway[s].items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if state == goal:
                    return path2
                else:
                    queue.append(path2)
    return []

In [231]:
shorter_path('鸡鸣寺','马群')

['鸡鸣寺',
 'line3',
 '浮桥',
 'line3',
 '大行宫',
 'line2',
 '西安门',
 'line2',
 '明故宫',
 'line2',
 '苜蓿园',
 'line2',
 '下马坊',
 'line2',
 '孝陵卫',
 'line2',
 '钟灵街',
 'line2',
 '马群']

for example, if you use Beijing subway graoh, and you run:

### (Optional) Improve your agent to make it able to find a path based on different strategies

###  <评阅点> : 是否正确得到不同目标下的路径。

Some ideas you might want to try:

a. Find the shortest path between two stations.

b. Find the path that requires minimum transfers between two stations.

c. Combine the previous two ideas, find a more suitable path.

Compare your results with results obtained by using some apps such as Baidu map, A map, Google map or Apple map. If there is difference, try to explanate it.

In [ ]:
import pandas
from bs4 import BeautifulSoup 
from urllib import request
import re
import pandas as pd
import numpy as np
import urllib.parse as urp
from xml.etree import ElementTree
import time

def __get_location1__(name,city):  
        my_ak = '9a5XmyAVTyunpTqZkv48c9ghnEFpswPa'    # 需要自己填写自己的AK
        tag = urp.quote('地铁站')
        qurey = urp.quote(name)
        try:
            url = 'http://api.map.baidu.com/place/v2/search?query='+qurey+'&tag='+'&region='+urp.quote(city)+'&output=json&ak='+my_ak
            print(url)
            req = request.urlopen(url)
            res = req.read().decode()
            lat = pd.to_numeric(re.findall('"lat":(.*)',res)[0].split(',')[0])
            lng = pd.to_numeric(re.findall('"lng":(.*)',res)[0])
            return (lng,lat)  #经度和纬度
        except:
            return 0,0

In [185]:
__get_location1__('鸡鸣寺','南京')

http://api.map.baidu.com/place/v2/search?query=%E9%B8%A1%E9%B8%A3%E5%AF%BA&tag=&region=%E5%8D%97%E4%BA%AC&output=json&ak=9a5XmyAVTyunpTqZkv48c9ghnEFpswPa


(118.80186100000002, 32.067125)

## Congratulations ! You have finished the assignment of week 2.

![title](img/agent.png)

### If you have any suggestions regarding the teaching, please feel free to send them to my eamil (eric.lee.xiao@gmail.com) 